In [ ]:
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
from scipy.optimize import least_squares
import scipy.constants
import matplotlib.pyplot as plt

: 

In [ ]:
f = 5.4e9
c = scipy.constants.c
wavelength = c/f
k = 2*np.pi*f/c
s = 0.0097
sbl = 0.013
theta = 40
theta_rad = np.deg2rad(theta)  

data = pd.read_csv('Merged_Data.csv')

trim_number = 3

mean_vv = data['VH'].mean()
std_vv = data['VH'].std()
data_trimmed = data[(data['VH'] >= mean_vv - trim_number * std_vv) & (data['VH'] <= mean_vv + trim_number * std_vv)]

VV_dB = data_trimmed['VV'].values
VH_dB = data_trimmed['VH'].values
SM = data_trimmed['SM'].values
LAI = data_trimmed['LAI'].values

# Convert dB to linear units
VV_linear = 10**(VV_dB / 10)
VH_linear = 10**(VH_dB / 10)
VH_VV_ratio = VH_linear / VV_linear

OH MODEL:

In [ ]:
def oh_vh_vv_ratio(a1, a3, a4, a5, a6):
    vh_vv_ratio = a1 * ((sbl + (np.sin(a3 * theta)))**a4) * (1 - np.exp(-a5 * ((k * s) ** a6)))
    return np.full_like(VH_VV_ratio, vh_vv_ratio)  

def oh_vh(moisture, A, B, C, D, E):

    sigma0_vh = (A *moisture**B *(np.cos(theta_rad))**C *(1 - np.exp(-D * (k*s)**E)))

    return sigma0_vh

WCM MODEL:

In [ ]:
def wcm_vh(LAI, A, B, C, sigma_soil):
    vegetation_term = A * (1 - np.exp(-B * LAI))
    soil_term = sigma_soil * np.exp(-2 * C * LAI / np.cos(theta_rad))
    sigma_vh = vegetation_term + soil_term
    return sigma_vh

WCM + OH MODEL:


DUBOIS MODEL:

In [ ]:
def dubois_model(epsilon_r, A, B):
    term1 = (np.sin(theta_rad) / wavelength)**2
    term2 = (epsilon_r - 1)**2
    term3 = np.cos(theta_rad)**3
    term4 = np.exp(-B * (k**2) * s**2 * np.cos(theta_rad)**2)
    sigma0_linear = A * term1 * term2 * term3 * term4
    sigma0_dB = 10 * np.log10(sigma0_linear)
    return np.full_like(VH_dB, sigma0_dB)

In [ ]:
# Optimization Functions

def optimize_oh_vh_vv_ratio():
    def residuals(params):
        predicted = oh_vh_vv_ratio(*params)
        return predicted - VH_VV_ratio

    initial_guess = [0.1, 1.3, 1.2, 0.9, 0.8]
    result = least_squares(residuals, initial_guess,  method='trf',  loss='soft_l1',  max_nfev=1000)
    return result.x


def optimize_oh_vh():
    def residuals(params):
        A, B, C, D, E = params
        predicted = oh_vh(SM, A, B, C, D, E)
        return predicted - VH_dB
    initial_guess = [0.11, 0.7, 2.2, 0.32, 1.8]
    result = least_squares(residuals, initial_guess)
    return result.x


def optimize_wcm():
    def residuals(params):
        A, B, C, sigma_soil = params
        predicted = wcm_vh(LAI, A, B, C, sigma_soil)
        return predicted - VH_dB
    initial_guess = [0.1, 0.1, 0.1, -15]  
    result = least_squares(residuals, initial_guess)
    return result.x


def optimize_dubois():
    epsilon_r = np.ones_like(VH_dB) * 5  

    def residuals(params):
        A, B = params
        predicted = dubois_model(epsilon_r, A, B)
        return predicted - VH_dB
    initial_guess = [0.1, 0.1]
    result = least_squares(residuals, initial_guess)
    return result.x


# Optimize and print results
oh_ratio_params = optimize_oh_vh_vv_ratio()
oh_vh_params = optimize_oh_vh()
wcm_params = optimize_wcm()
dubois_params = optimize_dubois()

#print("OH VH/VV ratio parameters (a1, a2):", oh_ratio_params)
print("OH VH parameters (A, B, C, D, E):", oh_vh_params)
print("WCM parameters (A, B, C, sigma_soil):", wcm_params)
print("Dubois parameters (A, B):", dubois_params)

# Calculate and print R² scores

In [ ]:
plt.scatter(SM, VH_dB)

In [ ]:
def calculate_r2_scores():
    # OH VH/VV ratio
    predicted_ratio = oh_vh_vv_ratio(*oh_ratio_params)
    r2_oh_ratio = r2_score(VH_VV_ratio, predicted_ratio)

    # OH VH
    predicted_vh = oh_vh(SM, *oh_vh_params)
    r2_oh_vh = r2_score(VH_dB, predicted_vh)

    # WCM
    predicted_wcm = wcm_vh(LAI, *wcm_params)
    r2_wcm = r2_score(VH_dB, predicted_wcm)

    # Dubois
    epsilon_r = np.ones_like(VH_dB) * 5
    wavelength = c / f
    predicted_dubois = dubois_model(epsilon_r, *dubois_params)
    r2_dubois = r2_score(VH_dB, predicted_dubois)

    # Print results
    print("\nR² scores:")
    print("OH Model VH/VV ratio:", r2_oh_ratio)
    print("OH Model VH:", r2_oh_vh)
    print("WCM Model:", r2_wcm)
    print("Dubois Model:", r2_dubois)


calculate_r2_scores()

In [ ]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.metrics import r2_score


def compute_sigma0_VH_model(LAI, mv, s, theta_rad, A, B, C, soil_constant, moisture_exponent, cosine_exponent, R_VH_base, R_VH_moisture_factor):
    ks = k * s

    # Soil contribution
    sigma0_HH_soil = (soil_constant * mv**moisture_exponent * np.cos(theta_rad)**cosine_exponent) * (1 - np.exp(-ks))

    R_VH = R_VH_base + R_VH_moisture_factor * mv

    sigma0_VH_soil = sigma0_HH_soil * R_VH

    # Vegetation contribution
    sigma0_VH = A * (1 - np.exp(-B * LAI)) + sigma0_VH_soil * np.exp(-C * LAI)

    return sigma0_VH


def residuals_wcm(params, LAI, mv, s, theta_rad, sigma0_VH_obs):
    sigma0_VH_pred = compute_sigma0_VH_model(LAI, mv, s, theta_rad, *params)
    return sigma0_VH_pred - sigma0_VH_obs


def fit_wcm_oh_model(LAI, mv, s, theta_rad, sigma0_VH_obs):
    # Initial guesses for all parameters
    initial_params = [0.1, 0.5, 0.3, 0.2, 0.6, 2.5, 0.1, 0.05]  # Includes soil_constant, moisture_exponent, etc.

    # Perform optimization
    result = least_squares(
        residuals_wcm,
        initial_params,
        args=(LAI, mv, s, theta_rad, sigma0_VH_obs),
        bounds=([0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 10, 1, 1]),  # Example bounds
        method="trf"
    )

    # Unpack optimized parameters
    A_opt, B_opt, C_opt, soil_constant_opt, moisture_exponent_opt, cosine_exponent_opt, R_VH_base_opt, R_VH_moisture_factor_opt = result.x

    # Compute predicted sigma0_VH
    sigma0_VH_pred = compute_sigma0_VH_model(
        LAI, mv, s, theta_rad, A_opt, B_opt, C_opt, soil_constant_opt, moisture_exponent_opt, cosine_exponent_opt, R_VH_base_opt, R_VH_moisture_factor_opt
    )

    # Calculate R²
    r2 = r2_score(sigma0_VH_obs, sigma0_VH_pred)

    # Print optimized parameters and R²
    print(f"Optimized parameters:")
    print(f"A = {A_opt:.4f}, B = {B_opt:.4f}, C = {C_opt:.4f}")
    print(f"Soil Constant = {soil_constant_opt:.4f}, Moisture Exponent = {moisture_exponent_opt:.4f}, Cosine Exponent = {cosine_exponent_opt:.4f}")
    print(f"R_VH Base = {R_VH_base_opt:.4f}, R_VH Moisture Factor = {R_VH_moisture_factor_opt:.4f}")
    print(f"R² = {r2:.4f}")

    return A_opt, B_opt, C_opt, soil_constant_opt, moisture_exponent_opt, cosine_exponent_opt, R_VH_base_opt, R_VH_moisture_factor_opt, r2, result


# Call the optimization function
A, B, C, soil_constant, moisture_exponent, cosine_exponent, R_VH_base, R_VH_moisture_factor, r2, fit_result = fit_wcm_oh_model(
    LAI, SM, s, theta_rad, VH_linear
)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Prepare the data
X = np.column_stack((LAI, SM))  # Input features: LAI and SM
y = np.column_stack((A_true, B_true, C_true, sigma_soil_true))  # True WCM parameters (if available)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4)  # Output layer for the 4 WCM parameters: A, B, C, sigma_soil
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

# Predict WCM parameters on the test set
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred, multioutput='uniform_average')
print(f"R² score for parameter prediction: {r2:.4f}")

# Use the predicted parameters in the WCM model
predicted_params = model.predict(X)  # Predict parameters for the entire dataset
A_pred, B_pred, C_pred, sigma_soil_pred = predicted_params.T

# Calculate WCM predictions using the predicted parameters


def wcm_vh(LAI, A, B, C, sigma_soil):
    vegetation_term = A * (1 - np.exp(-B * LAI))
    soil_term = sigma_soil * np.exp(-2 * C * LAI / np.cos(theta_rad))
    sigma_vh = vegetation_term + soil_term
    return sigma_vh


VH_pred = wcm_vh(LAI, A_pred, B_pred, C_pred, sigma_soil_pred)

# Calculate R² score for the WCM predictions
r2_wcm = r2_score(VH_linear, VH_pred)
print(f"R² score for WCM predictions: {r2_wcm:.4f}")